In [ ]:
import os
import yaml
import re
import math
import pickle
from typing import Any
from functools import cached_property

import gdsfactory as gf
from gdsfactory.generic_tech import get_generic_pdk
from gdsfactory.technology import LayerStack
from gplugins.common.base_models.component import LayeredComponentBase
from gplugins.tidy3d.util import get_port_normal, sort_layers

from skopt import gp_minimize
from skopt.plots import plot_convergence, plot_gaussian_process

import tidy3d as td
from tidy3d.components.geometry.base import from_shapely

from misc_tests.Optimization.device_opt.SimulationSettings import SimulationSettingsTiny3DFdtd, SIMULATION_SETTINGS_LUMERICAL_TINY3D_DEFAULT

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import numpy as np

import tiktoken
import openai
# Configure OpenAI API Key
OPENAI_API_KEY = "ENTER API KEY"
client = openai.OpenAI(api_key=OPENAI_API_KEY)

import pathlib
PathType = pathlib.Path | str

In [ ]:

tokenizer = tiktoken.get_encoding("cl100k_base")
def truncate_prompt(prompt, max_tokens=120000):
    # Tokenize the input prompt
    tokens = tokenizer.encode(prompt)
    
    # Check if the prompt exceeds the maximum allowed tokens
    if len(tokens) > max_tokens:
        # Truncate the prompt by keeping only the last `max_tokens` tokens
        tokens = tokens[-max_tokens:]
        
        # Decode tokens back to string
        truncated_prompt = tokenizer.decode(tokens)
        return truncated_prompt
    return prompt

def call_openai(prompt, sys_prompt='', model='gpt-4o'):
    '''
    calling openai. This is my persoanl account.
    '''
    prompt = truncate_prompt(prompt)
    
    #client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    response = client.chat.completions.create(
        # model="gpt-3.5-turbo",
        # model="gpt-4",
        # model="gpt-4-turbo",
        model=model,
        temperature= 0.1,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt},
        ]
    )

    return response.choices[0].message.content

def run_llm(SYS_PROMPT, USR_PROMPT):
    
    sys_prompt = SYS_PROMPT

    #print("################################################################################")
    #print("System Prompt Sent to OPENAI")
    #print("################################################################################")
    #print(sys_prompt)
    #print("################################################################################\n\n")
    
    usr_prompt = USR_PROMPT
    #print("################################################################################")
    #print("User Prompt Sent to OPENAI")
    #print("################################################################################")
    #print(data_to_send)
    #print("################################################################################\n\n")
    
    openai_response = call_openai(usr_prompt, sys_prompt)
    
    print("################################################################################")
    print("Response from OPENAI")
    print("################################################################################")
    print(openai_response)
    print("################################################################################\n\n")

    return openai_response

In [ ]:
# load intial user prompt from csv

SYS_PROMPT_INIT = 